### P1 Autoencoder NN

In [15]:
import numpy as np

In [16]:
def create_training_data():
    X=np.eye(8)
    y=X.copy()
    return X, y

In [25]:
class AutoencoderNetwork:
    def __init__(self, learning_rate=1.0, seed=42):
        np.random.seed(seed)
        self.learning_rate = learning_rate
        self.W1 = np.random.randn(8,3)*0.5
        self.b1 = np.random.randn(1,3)*0.5
        self.W2 = np.random.randn(3,8)*0.5
        self.b2 = np.random.randn(1,8)*0.5
        self.z1 = None
        self.a1 = None
        self.z2 = None
        self.a2 = None

        self.loss_history = []

    def sigmoid(self, z):
        z_val = np.clip(z, -500, 500)
        return 1/(1+np.exp(-z_val))

    def sigmoid_derivative(self, z):
        s = self.sigmoid(z)
        return s*(1-s)

    def forward_pass(self, X):
        self.z1 = X @ self.W1 + self.b1
        self.a1 = self.sigmoid(self.z1)
        self.z2 = self.a1 @ self.W2 + self.b2
        self.a2 = self.sigmoid(self.z2)

        return self.a2

    def compute_loss(self, output, target):
        m = output.shape[0]
        loss = np.mean((output-target)**2)/2
        return loss

    def backward_pass(self, X, y):
        m = X.shape[0]

        output_error = self.a2 - y
        delta2 = output_error * self.sigmoid_derivative(self.z2)

        hidden_error = delta2 @ self.W2.T
        delta1 = hidden_error * self.sigmoid_derivative(self.z1)

        dw2 = (self.a1.T @ delta2) / m
        db2 = np.mean(delta2, axis=0, keepdims=True)

        dw1 = (X.T @ delta1) / m
        db1 = np.mean(delta1, axis=0, keepdims=True)

        return dw1, db1, dw2, db2

    def update_weights(self, dw1, db1, dw2, db2):
        self.W1 -= self.learning_rate * dw1
        self.b1 -= self.learning_rate * db1
        self.W2 -= self.learning_rate * dw2
        self.b2 -= self.learning_rate * db2

In [26]:
def train_network(network, X, y, epochs=5000):
    for epoch in range(epochs):
        output = network.forward_pass(X)

        loss = network.compute_loss(output, y)
        network.loss_history.append(loss)

        dw1, db1, dw2, db2 = network.backward_pass(X, y)

        network.update_weights(dw1, db1, dw2, db2)

        if epoch % 1000 == 0:
            print(f"Epoch {epoch:4d}: Loss = {loss:.6f}")

    print(f"\n Training complete, final loss: {loss:.6f}")
    return network

In [27]:
def analyze_results(network, X, y):
    output = network.forward_pass(X)
    hidden = network.a1

    print("\nReconstruction Results:")
    print("-" * 60)
    print("Input Pattern -> Hidden Values -> Output Pattern")
    print("-" * 60)
    
    for i in range(len(X)):
        input_str = ''.join([str(int(x)) for x in X[i]])
        hidden_str = '[' + ', '.join([f"{h:.3f}" for h in hidden[i]]) + ']'
        output_str = ''.join([str(int(round(x))) for x in output[i]])
        print(f"{input_str} -> {hidden_str} -> {output_str}")
        
    # Calculate reconstruction accuracy
    predictions_rounded = np.round(output)
    accuracy = np.mean(predictions_rounded == y) * 100
    print(f"\nReconstruction Accuracy: {accuracy:.1f}%")
    
    return hidden

In [28]:
if __name__ == "__main__":
    X, y = create_training_data()

    network = AutoencoderNetwork(learning_rate=1.0)

    network = train_network(network, X, y, epochs=5000)
    
    hidden_representations = analyze_results(network, X, y)

Epoch    0: Loss = 0.126346
Epoch 1000: Loss = 0.024064
Epoch 2000: Loss = 0.010835
Epoch 3000: Loss = 0.008450
Epoch 4000: Loss = 0.006754

 Training complete, final loss: 0.003289

Reconstruction Results:
------------------------------------------------------------
Input Pattern -> Hidden Values -> Output Pattern
------------------------------------------------------------
10000000 -> [0.589, 0.028, 0.990] -> 10000000
01000000 -> [0.927, 0.975, 0.026] -> 01000000
00100000 -> [0.979, 0.802, 0.885] -> 00100000
00010000 -> [0.080, 0.025, 0.034] -> 00010000
00001000 -> [0.952, 0.026, 0.095] -> 00001000
00000100 -> [0.212, 0.986, 0.987] -> 00000100
00000010 -> [0.006, 0.217, 0.864] -> 00000010
00000001 -> [0.017, 0.938, 0.081] -> 00000001

Reconstruction Accuracy: 100.0%


### B) Purpose of training algorithm in Autoencoder:

The training algorithm teaches the network on how to compress information and then decompress it correctly. the autoencoder is like a bottleneck where, encoder (input->hidden) : here the network must squeeze 8 values down into 3 values then decoder (hidden -> output): must then expand those 3 values into the original 8 values. this helps the network learn which features are truly important, with only 3 neurons in the middle, encoder cannot store all information directly so it must find a encoding scheme. During decoder phase that is backpropagation, network discovers that each pattern can be represented as a unique combination of three hidden neurons. So the 3 hidden neurons contain a compressed representation that captures essential information and this helps to accurately rebuild the input. 